# **News Web Scraping and Recommendations Using GPT2**

**Install Libraries**

In [1]:
!pip install newsapi-python


In [2]:
!pip install pycountry


In [3]:
!pip install transformers


**Import Libraires**

In [4]:
import requests
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
import pandas as pd
import torch
import warnings
import contextlib
from transformers import GPT2LMHeadModel, GPT2Tokenizer


**Web Scraping and Data Collection**

In [5]:
# Initialize the News API client with your API key
api_key = '7e3c6c7cfc3c4a739569b5f04f138ab0'  #API key for access update news data
newsapi = NewsApiClient(api_key=api_key)


In [6]:
# Define the news sources you want to fetch data from
news_sources = ['the-times-of-india', 'bbc-news', 'aajtak', 'cnn']

# Create a dictionary to store news data for each source
news_data = {}

# Iterate through the news sources
for source in news_sources:
    try:
        # Use the News API to fetch top headlines from the specified source
        top_headlines = newsapi.get_top_headlines(sources=source, language='en')

        # Retrieve the headlines' data
        headlines = top_headlines['articles']

        if headlines:
            # Format and store the news articles for the source
            formatted_headlines = []
            for article in headlines:
                formatted_article = {
                    "date": article['publishedAt'],  # Add the date field
                    "title": article['title'],
                    "description": article['description'],
                    "url": article['url'],
                    "source": article['source']['name'],
                }
                formatted_headlines.append(formatted_article)

            news_data[source] = formatted_headlines

    except Exception as e:
        print(f"An error occurred while fetching news from {source}: {str(e)}")
print(news_data)


{'the-times-of-india': [{'date': '2021-04-23T08:29:00Z', 'title': 'Army explores procurement of 350 light tanks for mountainous terrain after border standoff with China', 'description': 'India News:  The Army is now exploring the possibility of procuring 350 light tanks, which can also be transported by air, to augment its firepower in high-altitu', 'url': 'http://timesofindia.indiatimes.com/india/army-explores-procurement-of-350-light-tanks-for-mountainous-terrain-after-border-standoff-with-china/articleshow/82217825.cms', 'source': 'The Times of India'}, {'date': '2021-04-23T05:44:49Z', 'title': 'PBKS vs MI Live Score, IPL 2021: Mumbai Indians seek consistency; Punjab Kings eye return to winning ways', 'description': 'IPL Live Score: Mumbai Indians seek consistency; Punjab Kings eye return to winning ways. Stay with TOI to get IPL live score, playing 11, scorecard, highlights and ball by ball score updates of the 17th IPL match between Punjab Kings and Mumbai Indians.', 'url': 'http:

**Data Transformation and Pandas DataFrame**

In [8]:
news_data
type(news_data)


dict

In [9]:
# Create a list to store all the news articles
all_articles = []

# Iterate through the sources and their respective articles
for source, articles in news_data.items():
    for article in articles:
        # Add the source as an additional field
        article["source"] = source
        all_articles.append(article)

# Convert the list of dictionaries into a Pandas DataFrame
df = pd.DataFrame(all_articles)

# Display the DataFrame
print(df)


                            date  \
0           2021-04-23T08:29:00Z   
1           2021-04-23T05:44:49Z   
2           2021-04-23T05:43:00Z   
3           2021-04-23T04:41:00Z   
4           2021-04-23T04:23:00Z   
5           2021-04-23T03:54:00Z   
6           2021-04-22T16:20:06Z   
7     2024-08-14T16:52:14.22471Z   
8   2024-08-14T16:07:20.1307007Z   
9   2024-08-14T16:07:18.1611064Z   
10  2024-08-14T15:52:17.3016894Z   
11  2024-08-14T13:52:17.6315781Z   
12  2024-08-14T13:22:15.7924018Z   
13  2024-08-14T12:52:19.7348057Z   
14  2024-08-14T10:37:26.5468668Z   
15  2024-08-14T10:37:23.2664328Z   
16  2024-08-14T03:07:12.8444767Z   
17      2024-08-14T16:01:52.966Z   
18          2024-08-14T14:43:39Z   
19          2024-08-14T11:33:56Z   
20          2024-08-14T11:00:54Z   
21          2024-08-14T09:42:16Z   
22          2024-08-14T09:00:03Z   
23      2024-08-14T02:31:50.411Z   
24       2024-08-13T21:35:37.09Z   
25          2024-08-13T19:33:49Z   
26      2024-08-13T18:47:45.

In [10]:
df


,date,title,description,url,source
0,2021-04-23T08:29:00Z,Army explores procurement of 350 light tanks f...,India News: The Army is now exploring the pos...,http://timesofindia.indiatimes.com/india/army-...,the-times-of-india
1,2021-04-23T05:44:49Z,"PBKS vs MI Live Score, IPL 2021: Mumbai Indian...",IPL Live Score: Mumbai Indians seek consistenc...,http://timesofindia.indiatimes.com/sports/cric...,the-times-of-india
2,2021-04-23T05:43:00Z,Zydus Cadila gets DCGI nod for hepatitis drug ...,India News: Drug firm Zydus Cadila on Friday s...,http://timesofindia.indiatimes.com/india/zydus...,the-times-of-india
3,2021-04-23T04:41:00Z,Even record death toll may hide extent of Indi...,India News: Bodies piling up at crematoriums a...,http://timesofindia.indiatimes.com/india/even-...,the-times-of-india
4,2021-04-23T04:23:00Z,Government to provide 5 kg free food grains to...,India News: The government on Friday announced...,http://timesofindia.indiatimes.com/india/gover...,the-times-of-india
5,2021-04-23T03:54:00Z,'Inappropriate': PM Modi objects to 'protocol ...,India News: Delhi chief minister Arvind Kejriw...,http://timesofindia.indiatimes.com/india/delhi...,the-times-of-india
6,2021-04-22T16:20:06Z,Coronavirus in India live updates: Zydus' Vira...,"In yet another grim milestone, India recorded ...",http://timesofindia.indiatimes.com/india/coron...,the-times-of-india
7,2024-08-14T16:52:14.22471Z,London stabbing victim is 11-year-old Australi...,The girl was in the UK capital on holiday when...,https://www.bbc.co.uk/news/articles/cgm71zynrm9o,bbc-news
8,2024-08-14T16:07:20.1307007Z,Trump to tout economic plan after union rights...,Trump will speak in North Carolina after the U...,https://www.bbc.co.uk/news/articles/c8xlq0rg4yzo,bbc-news
9,2024-08-14T16:07:18.1611064Z,Stonehenge: Central Altar Stone from Scotland ...,Stonehenge's famous Altar Stone came from Scot...,https://www.bbc.co.uk/news/articles/c207lqdn755o,bbc-news


**Recommendation Generation Using GPT2**

In [11]:
# Load the GPT-2 model and tokenizer
model_name = "gpt2"  # You can use "gpt2-medium" or other variants for different sizes
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\amamo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amamo\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

c:\Users\amamo\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
def generate_recommendations(prompt, max_length=100):
    # Tokenize the prompt and generate text
    input_ids = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)

    # Suppress the warning messages
    with warnings.catch_warnings(), contextlib.redirect_stderr(None):
        warnings.simplefilter("ignore")
        outputs = model.generate(input_ids, max_length=max_length, no_repeat_ngram_size=2, num_return_sequences=1, do_sample=False)

    # Decode and return the recommendation
    recommendation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return recommendation


In [13]:
# Example usage with your DataFrame
for index, row in df.iterrows():
    user_prompt = f"Please recommend a news article about {row['title']} from {row['source']} with the following description: {row['description']}"
    recommendation = generate_recommendations(user_prompt)
    print(f"Recommendation for {row['title']} ({row['source']}):\n{recommendation}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recommendation for Army explores procurement of 350 light tanks for mountainous terrain after border standoff with China (the-times-of-india):
Please recommend a news article about Army explores procurement of 350 light tanks for mountainous terrain after border standoff with China from the-times-of-india with the following description: India News:  The Army is now exploring the possibility of procuring 350 light tanks, which can also be transported by air, to augment its firepower in high-altitu terrain. The Indian Army has been conducting a series of high altitude reconnaissance missions over the Himalayas, and has conducted reconnaissance flights over India's border



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recommendation for PBKS vs MI Live Score, IPL 2021: Mumbai Indians seek consistency; Punjab Kings eye return to winning ways (the-times-of-india):
Please recommend a news article about PBKS vs MI Live Score, IPL 2021: Mumbai Indians seek consistency; Punjab Kings eye return to winning ways from the-times-of-india with the following description: IPL Live Score: Mumbai Indians seek consistency; Punjab Kings eye return to winning ways. Stay with TOI to get IPL live score, playing 11, scorecard, highlights and ball by ball score updates of the 17th IPL match between Punjab Kings and Mumbai Indians.




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Recommendation for Kaitlan Collins describes her Uber driver’s reaction to hearing her talking with Trump | CNN Politics (cnn):
Please recommend a news article about Kaitlan Collins describes her Uber driver’s reaction to hearing her talking with Trump | CNN Politics from cnn with the following description: CNN’s Kaitlan Collins shares her experience of catching an Uber driver off guard by calling former President Donald Trump from the backseat of his vehicle.

"I was driving my car and I heard a guy say, 'Hey, you're a Trump supporter,'" Collins said. "I said, "Well



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recommendation for Video: Key Trump allies want him to stop insulting Harris | CNN (cnn):
Please recommend a news article about Video: Key Trump allies want him to stop insulting Harris | CNN from cnn with the following description: CNN political commentator SE Cupp discusses why former President Donald Trump’s allies, including Peter Navarro and Kellyanne Conway, are urging him to stop insulting Kamala Harris. Cupp argues that such comments could negatively impact his support, especiall… https://t.co/9QXQzQ9X9o — CNN Politics



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recommendation for With ‘Cancer Moonshot’ announcement, Biden turns to causes most important to him in final months in office (cnn):
Please recommend a news article about With ‘Cancer Moonshot’ announcement, Biden turns to causes most important to him in final months in office from cnn with the following description: President Joe Biden’s first public event planned since his exit from the 2024 race last month marked a cause close to home: Announcing a major financial award from the cancer-curing policy arm he created in the wake of his son Beau Biden’s death. The announcement was made on the

Recommendation for House blasted with water after contractor’s mistake | CNN (cnn):
Please recommend a news article about House blasted with water after contractor’s mistake | CNN from cnn with the following description: A home in Edison, New Jersey was deluged with water after an apparent contractor mistake.

The water was found in a home on the 400 block of West Main Street, according to the Edi